In [1]:
import re
import joblib
import feather
import numpy as np
import pandas as pd

from datetime import date
from currency_converter import CurrencyConverter

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline

In [2]:
def get_df(name):
    df = pd.read_excel('{0}'.format(name),
                       dtype={
                           'FOLIO_RUPC': str, 'CLAVEUC': str, 'CODIGO_CONTRATO': str,
                           'CODIGO_EXPEDIENTE': str, 'FOLIO_RUPC': str
                       }
                      )
    df.loc[:, 'PROC_F_PUBLICACION'] = pd.to_datetime(df.PROC_F_PUBLICACION, yearfirst=True)
    df.loc[:, 'FECHA_APERTURA_PROPOSICIONES'] = pd.to_datetime(df.FECHA_APERTURA_PROPOSICIONES, yearfirst=True)
    df.loc[:, 'EXP_F_FALLO'] = pd.to_datetime(df.EXP_F_FALLO, yearfirst=True)
    df.loc[:, 'FECHA_CELEBRACION'] = pd.to_datetime(df.FECHA_CELEBRACION, yearfirst=True)
    df.loc[:, 'FECHA_INICIO'] = pd.to_datetime(df.FECHA_INICIO, yearfirst=True)
    df.loc[:, 'FECHA_FIN'] = pd.to_datetime(df.FECHA_FIN, yearfirst=True)
    cols = [
        'ESTRATIFICACION_MUC', 'NOMBRE_DE_LA_UC', 'RESPONSABLE', 'DEPENDENCIA', 'PROVEEDOR_CONTRATISTA',
        'ESTRATIFICACION_MPC', 'TITULO_EXPEDIENTE', 'TITULO_CONTRATO', 'TIPO_CONTRATACION',
        'TIPO_PROCEDIMIENTO', 'ESTATUS_EMPRESA', 'ESTATUS_CONTRATO', 'SIGLAS'
    ]
    for col in cols:
        df.loc[:, col] = df[col].str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.upper()
        df.loc[:, col] = df[col].str.strip()

    df.loc[:, 'NUMERO_PROCEDIMIENTO'] = df.NUMERO_PROCEDIMIENTO.str.upper()
    return df

def convert_to_mxn(montos, monedas, fechas):
    monto_pesos = []
    for monto, moneda, fecha in zip(montos, monedas, fechas):
        if moneda in {'MXN', 'TEST', 'OTH'}:
            monto_pesos.append(monto)
        else:
            val = converter.convert(monto, moneda, 'MXN', date=fecha)
            monto_pesos.append(val)
    return monto_pesos


def get_claveuc_proc(num_proc):
    if isinstance(num_proc, str):
        return num_proc.split('-')[1]
    return None
    

def get_claveuc_nombre(nombre_uc):
    if isinstance(nombre_uc, str):
        nombre_uc =  nombre_uc.split('#')
    else:
        return None
    if len(nombre_uc) > 1 and isinstance(nombre_uc, list):
        return nombre_uc[1]
    return None


def get_claveuc_real(df, unidades_validas):
    cond1 = (df.CLAVEUC == df.CLAVEUC_PROC)
    cond2 = (df.CLAVEUC == df.CLAVEUC_NOM)
    cond3 = (df.CLAVEUC_NOM == df.CLAVEUC_PROC)
    df_aux = df.assign(CLAVEUC_REAL='MISSING')
    # Set the value of clave if at least one condition is met
    df_aux.loc[cond1[cond1 == True].index, 'CLAVEUC_REAL'] = df_aux.loc[cond1[cond1 == True].index, 'CLAVEUC']
    df_aux.loc[cond2[cond2 == True].index, 'CLAVEUC_REAL'] = df_aux.loc[cond2[cond2 == True].index, 'CLAVEUC']
    df_aux.loc[cond3[cond3 == True].index, 'CLAVEUC_REAL'] = df_aux.loc[cond3[cond3 == True].index, 'CLAVEUC_PROC']
    
    df_test = df_aux.loc[df_aux.CLAVEUC_REAL == 'MISSING']
    cond_nom = df_test.CLAVEUC_NOM.map(lambda clave: clave in unidades_validas)
    cond_proc = df_test.CLAVEUC_PROC.map(lambda clave: clave in unidades_validas)
    cond_clave = df_test.CLAVEUC.map(lambda clave: clave in unidades_validas)
    
    df_aux.loc[
        cond_nom[cond_nom == True].index, 'CLAVEUC_REAL'
    ] = df_aux.loc[cond_nom[cond_nom == True].index].CLAVEUC_NOM

    df_aux.loc[
        cond_proc[cond_proc == True].index, 'CLAVEUC_REAL'
    ] = df_aux.loc[cond_proc[cond_proc == True].index].CLAVEUC_PROC

    df_aux.loc[
        cond_clave[cond_clave == True].index, 'CLAVEUC_REAL'
    ] = df_aux.loc[cond_clave[cond_clave == True].index].CLAVEUC
    
    return df_aux

In [3]:
file_names = [
    '../data/raw/Procedimiento de Contratación 2010-2012.xlsx',
    '../data/raw/Procedimiento de Contratación 2013.xlsx',
    '../data/raw/Procedimiento de Contratación 2014.xlsx',
    '../data/raw/Procedimiento de Contratación 2015.xlsx',
    '../data/raw/Procedimiento de Contratación 2016.xlsx'
]

df_procs = joblib.Parallel(n_jobs=-1, batch_size=1)(joblib.delayed(get_df)(name) for name in file_names)
df_procs = pd.concat(df_procs, axis=0, ignore_index=True)
df_procs = df_procs.loc[(df_procs.FECHA_INICIO.dt.year > 2011), :]

print(df_procs.shape)

ordered_cols = [
    'GOBIERNO',
    # Dependencia/unidad/responsable
    'DEPENDENCIA',
    'SIGLAS',    
    'NOMBRE_DE_LA_UC',
    'CLAVEUC',
    'RESPONSABLE',
    'ESTRATIFICACION_MUC',
    # proveedor
    'FOLIO_RUPC',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_EMPRESA',
    'ESTRATIFICACION_MPC',
    'CLAVE_CARTERA_SHCP',
    # montos
    'IMPORTE_CONTRATO',
    'APORTACION_FEDERAL',
    'MONEDA',
    # datos procedimiento
    'NUMERO_PROCEDIMIENTO',
    'FORMA_PROCEDIMIENTO',
    'TIPO_PROCEDIMIENTO',
    # datos contrato
    'CODIGO_CONTRATO',
    'TITULO_CONTRATO',
    'IDENTIFICADOR_CM',
    'TIPO_CONTRATACION',
    'ESTATUS_CONTRATO',
    'COMPRA_CONSOLIDADA',
    'PLURIANUAL',
    'CARACTER',
    'CONTRATO_MARCO',
    'CONVENIO_MODIFICATORIO',

    # Fechas ordenadas
    'PROC_F_PUBLICACION',
    'FECHA_APERTURA_PROPOSICIONES',
    'EXP_F_FALLO',
    'FECHA_CELEBRACION',
    'FECHA_INICIO',
    'FECHA_FIN',
    # expediente
    'CODIGO_EXPEDIENTE',
    'TITULO_EXPEDIENTE',
    'PLANTILLA_EXPEDIENTE',
    # otros
    'CLAVE_PROGRAMA',
    'CUENTA_ADMINISTRADA_POR',
    'ANUNCIO',
    'ARCHIVADO',
    'SIGLAS_PAIS',
    'RAMO',
    'ORGANISMO',
    'C_EXTERNO',
]

df_procs = df_procs.loc[:, ordered_cols]

df_procedimientos = df_procs

print(df_procedimientos.shape)

(861169, 45)
(861169, 45)


In [4]:
dep_repetidas = {
    'HOSPITAL GENERAL DE MEXICO "DR. EDUARDO LICEAGA"': 'HOSPITAL GENERAL DE MEXICO',
    'INSTITUTO NACIONAL DE REHABILITACION LUIS GUILLERMO IBARRA IBARRA': 'INSTITUTO NACIONAL DE REHABILITACION',
    'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA, "ING. JORGE L. TAMAYO", A.C.': 'CENTRO DE INVESTIGACION EN GEOGRAFIA Y GEOMATICA "ING. JORGE L. TAMAYO", A.C.',
    'TRIBUNAL FEDERAL DE JUSTICIA FISCAL Y ADMINISTRATIVA': 'TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA',
    'INSTITUTO DE INVESTIGACIONES ELECTRICAS': 'INSTITUTO NACIONAL DE ELECTRICIDAD Y ENERGIAS LIMPIAS'
}

for k, v in dep_repetidas.items():
    df_procedimientos.loc[df_procedimientos.DEPENDENCIA == k, 'DEPENDENCIA'] = v

del dep_repetidas

# convertir a Pesos
converter = CurrencyConverter(fallback_on_missing_rate=True)
monto_en_pesos = convert_to_mxn(
    df_procedimientos.IMPORTE_CONTRATO.values, df_procedimientos.MONEDA.values, df_procedimientos.FECHA_INICIO)

df_procedimientos = df_procedimientos.assign(IMPORTE_PESOS=monto_en_pesos)

df_procedimientos = df_procedimientos.assign(CLAVEUC_PROC=df_procedimientos.NUMERO_PROCEDIMIENTO.map(get_claveuc_proc))
df_procedimientos = df_procedimientos.assign(CLAVEUC_NOM=df_procedimientos.NOMBRE_DE_LA_UC.map(get_claveuc_nombre))

del monto_en_pesos

# Limpiar CLAVEUC
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 8, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')
df_procedimientos.loc[
    df_procedimientos.CLAVEUC.str.len() == 7, 'CLAVEUC'] = df_procedimientos.CLAVEUC.str.pad(9, fillchar='0')

# limpiar PoC
df_procedimientos.loc[:, 'PROVEEDOR_CONTRATISTA'] = df_procedimientos.PROVEEDOR_CONTRATISTA.str.replace('.', '')
df_procedimientos.loc[:, 'PROVEEDOR_CONTRATISTA'] = df_procedimientos.PROVEEDOR_CONTRATISTA.str.replace(',', '')
df_procedimientos.loc[:, 'PROVEEDOR_CONTRATISTA'] = df_procedimientos.PROVEEDOR_CONTRATISTA.str.replace('"', '')
df_procedimientos.loc[:, 'PROVEEDOR_CONTRATISTA'] = df_procedimientos.PROVEEDOR_CONTRATISTA.str.replace("'", '')

unidades_directorio = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls', dtype=str)
unidades_directorio = set(
    unidades_directorio.loc[:, 'Clave UC en CompraNet'].unique())

unidades_est = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Estatal.xls', dtype=str)
unidades_est = set(
    unidades_est.loc[:, 'Clave UC en CompraNet'].unique())

unidades_mun = pd.read_excel(
    '../data/raw/Directorio Unidades Compradoras Gobierno Municipal.xls', dtype=str)
unidades_mun = set(
    unidades_mun.loc[:, 'Clave UC en CompraNet'].unique())

unidades_directorio = unidades_directorio | unidades_est | unidades_mun
print(len(unidades_directorio))

print(df_procedimientos.shape)

df_procedimientos = get_claveuc_real(df_procedimientos, unidades_directorio)


5181
(861169, 48)


In [10]:
# Guardar

df_procedimientos = df_procedimientos.drop(['CLAVE_CARTERA_SHCP', 'CLAVEUC_PROC', 'CLAVEUC_NOM'], axis=1)
feather.write_dataframe(df_procedimientos, '../data/processed/procedimientos_all_2017_06_28.feather')
df_procedimientos.to_csv(
    '../data/processed/procedimientos_all_2017_06_28.psv', sep='|', encoding='utf-8', index=False, quoting=1)

In [19]:
# df_test = feather.read_dataframe('../data/processed/procedimientos_all_2017_06_28.feather')
df_test = pd.read_csv('../data/processed/procedimientos_all_2017_06_28.psv', sep='|')
print(df_test.shape)
df_test.head()

/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19,36,38,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(861169, 46)


,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS,CLAVEUC_REAL
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,208993,N44 MOB ETC,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23 00:00:00,NaN,2012-10-23 00:00:00,2012-12-31 00:00:00,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,512930.40,901024986
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,208992,N44 MOB ETC,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23 00:00:00,NaN,2012-10-23 00:00:00,2012-10-31 00:00:00,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,1887409.40,901024986
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,NaN,PAPELERIA CONSUMIBLES Y ACCESORIOS SA DE CV,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,NaN,INVITACION A CUANDO MENOS 3 PERSONAS,257525,IEA.DJ.CONT. 1140/2012,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19 00:00:00,NaN,2012-11-12 00:00:00,2012-12-31 00:00:00,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04. Invitación a cuando menos tres N...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,1892535.82,901024986
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761.0,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,NaN,ADJUDICACION DIRECTA FEDERAL,241054,CONSUMIBLES,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-09-06 08:49:00,2012-09-07 09:00:00,NaN,NaN,2012-09-11 00:00:00,2012-12-31 00:00:00,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07. Adjudicación Directa Nacional Ar...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,24234.00,901024986
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761.0,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,NaN,ADJUDICACION DIRECTA FEDERAL,474294,ADQUISICION DE EQUIPO DE COMPUTO,NaN,ADQUISICIONES,EXPIRADO,0.0,0.0,NaN,0.0,0.0,2012-12-12 11:38:00,2012-12-14 10:00:00,NaN,NaN,2012-12-17 00:00:00,2012-12-31 00:00:00,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07. Adjudicación Directa Nacional Ar...,NaN,PoC,NaN,No,MX,NaN,NaN,NaN,18700.00,901024986


## Leer solo Gobierno federal

In [ ]:
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)
print(df_procedimientos.shape)

## Tirar duplicados

In [8]:
cols_id = ['NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO', 'CODIGO_EXPEDIENTE']
df_validos = df_procedimientos.drop_duplicates(subset=cols_id, keep='first')
print(df_validos.shape)

(860631, 49)


In [ ]:
# clean_cols = [
#     'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO', 'CODIGO_EXPEDIENTE',
#     'ESTATUS_CONTRATO', 'DEPENDENCIA', 'SIGLAS', 'NOMBRE_DE_LA_UC'
# ]

# for col in clean_cols:
#     df_procedimientos.loc[:, col] = df_procedimientos[col].str.strip()

## Save

# df_procedimientos.drop(
#     'CLAVE_CARTERA_SHCP', axis=1
# ).to_csv('../data/processed/procedimientos_2017_06_14.csv', index=False, quoting=1, encoding='utf-8', sep='|')

# feather.write_dataframe(
#     df_procedimientos.drop('CLAVE_CARTERA_SHCP', axis=1), '../data/processed/procedimientos_2017_06_14.feather')

# df_procedimientos.head()